# CARLA Python API - Lab2 

This notebook mirrors `notebook.ipynb` and includes worked solutions for all exercises.

Official documentation: [CARLA Python API](https://carla.readthedocs.io/en/latest/python_api/).

In [14]:
import carla
import time

## 1. Connect to the CARLA server

The first step is creating the `client`, which is the remote access point to the simulator.

`carla.Client(host, port)` does not start a new simulation: it connects to an already running CARLA server.

> Note: `Client` is an RPC proxy. Every API call sends a request to the server and receives a response.

In [ ]:
client = carla.Client("localhost", 2000)

The connection is not immediate: network handshake, world loading, and latency can take a few seconds.

For this reason, set a suitable timeout before expensive operations such as `load_world`.

In [ ]:
client.set_timeout(0.5)
client.load_world("Town01") # a time consuming operation

In [ ]:
client.set_timeout(15)
client.load_world("Town02")

In [ ]:
print(f"Client version: {client.get_client_version()}")
print(f"Server version: {client.get_server_version()}")

## 2. Configure the simulation

After connecting, the `client` can manage the simulation context: map, weather, and other global settings.

### Change map

Loading a new map requires a full world reload. Waiting a few seconds is expected.

Additional maps can be downloaded from [GitHub Releases](https://github.com/carla-simulator/carla/releases).

Useful references:
- [Maps and navigation](https://carla.readthedocs.io/en/latest/core_map/)
- [`carla.Client.get_available_maps`](https://carla.readthedocs.io/en/latest/python_api/#carla.Client.get_available_maps)
- [`carla.Client.load_world`](https://carla.readthedocs.io/en/latest/python_api/#carla.Client.load_world)

In [ ]:
client.get_available_maps()

`Town10HD_Opt` e una versione alleggerita di `Town10HD`: mantiene buon dettaglio visivo con un costo computazionale piu basso.

In laboratorio e spesso una scelta pratica quando si vogliono test rapidi e stabili.

In [ ]:
client.load_world("Town10HD_Opt")

### Set weather

`carla.WeatherParameters` includes presets that are useful for repeatable tests. Changing weather affects lighting, reflections, and visibility, and can therefore affect sensor perception as well.

Useful references:
- [Weather (core world)](https://carla.readthedocs.io/en/latest/core_world/#weather)
- [`carla.WeatherParameters`](https://carla.readthedocs.io/en/latest/python_api/#carla.WeatherParameters)
- [`carla.World.set_weather`](https://carla.readthedocs.io/en/latest/python_api/#carla.World.set_weather)

In [36]:
world = client.get_world()
world.set_weather(carla.WeatherParameters.WetNight)

In [ ]:
weathers = [
    carla.WeatherParameters.CloudyNoon,
    carla.WeatherParameters.MidRainSunset,
    carla.WeatherParameters.HardRainNoon,
    carla.WeatherParameters.ClearSunset,
    carla.WeatherParameters.SoftRainNight,
    carla.WeatherParameters.Default,
]

for weather in weathers:
    world.set_weather(weather)
    print(f"Weather set to {weather}")
    time.sleep(5)

### Synchronous vs asynchronous mode

In **synchronous** mode, the server advances only when the client calls `world.tick()`. This is useful for controlled experiments and reproducible results.

In **asynchronous** mode, the server advances on its own at the highest possible speed. This is useful for quick demos, but frame-by-frame control is lower.

Practical rule:
- use **synchronous** mode when you need to coordinate logic, sensors, or data collection;
- use **asynchronous** mode when you mainly want to observe the simulation in real time.

Useful references:
- [Synchrony and timestep](https://carla.readthedocs.io/en/latest/adv_synchrony_timestep/)
- [`carla.WorldSettings`](https://carla.readthedocs.io/en/latest/python_api/#carla.WorldSettings)
- [`carla.World.tick`](https://carla.readthedocs.io/en/latest/python_api/#carla.World.tick)

![Sync vs Async](./sync.png)

In [ ]:
world = client.get_world()
settings = world.get_settings()
settings.synchronous_mode = True
world.apply_settings(settings)

In [ ]:
count = 0
while True:
    world.tick()
    time.sleep(0.1)
    count += 1
    print(f"Ticked the server {count}", end="\r")

In [ ]:
settings.synchronous_mode = False
world.apply_settings(settings)
client.reload_world()

## 3. Configuration exercises

These exercises are designed to consolidate the concepts above without introducing new topics.

### Guided exercise

Implement a script named `scenario_config.py` that:
1. connects to the server;
2. loads `Town10HD_Opt`;
3. applies at least 4 weather presets in sequence (with a 3-5 second pause);
4. switches to synchronous mode for 100 ticks;
5. switches back to asynchronous mode and reloads the world.

Required output: print each state transition to console (map, weather, mode, tick count).

### Additional exercises

1. Timeout handling: test `set_timeout(0.5)`, `5`, `15` and report which operations fail or succeed.
2. Sync/async comparison: measure how many ticks happen in 10 seconds in both modes.
3. Weather sequence: create a function that receives a list of presets and applies them in a continuous loop.

## 4. Add actors to the simulation

### Coordinate system

Actors use `carla.Transform`, made of:
- `Location(x, y, z)` in meters;
- `Rotation(pitch, yaw, roll)` in degrees.

Understanding position and orientation is essential: small errors on `yaw` or `z` can cause invalid spawns or poor camera viewpoints.

In [27]:
spectator = world.get_spectator()
spectator.set_transform(
    carla.Transform(
        carla.Location(x=0, y=10, z=0),
        carla.Rotation(yaw=0, pitch=0, roll=0)
    )
)

### Spawn points

Spawn points are predefined map transforms where it is recommended to place actors (vehicles, pedestrians, sensors).

Cycling through them with the spectator is a quick way to understand scene geometry before spawning real actors.

Useful references:
- [`carla.Map.get_spawn_points`](https://carla.readthedocs.io/en/latest/python_api/#carla.Map.get_spawn_points)
- [`carla.World.get_map`](https://carla.readthedocs.io/en/latest/python_api/#carla.World.get_map)

In [ ]:
world = client.get_world()
spawn_points = world.get_map().get_spawn_points()
spectator = world.get_spectator()

for spawn_point in spawn_points:
    spectator.set_transform(spawn_point)

    print(spawn_point.location, end="\r")
    time.sleep(5)

### Blueprints and actors

The blueprint library contains the available actor models. Each blueprint has an `id` and `tags` that are useful for filtering what you need.

Useful references:
- [`carla.World.get_blueprint_library`](https://carla.readthedocs.io/en/latest/python_api/#carla.World.get_blueprint_library)
- [`carla.BlueprintLibrary`](https://carla.readthedocs.io/en/latest/python_api/#carla.BlueprintLibrary)
- [`carla.ActorBlueprint`](https://carla.readthedocs.io/en/latest/python_api/#carla.ActorBlueprint)

In [ ]:
blue_prints = world.get_blueprint_library().filter('*')

for blue_print in blue_prints:
    print(f"Id: {blue_print.id}, Tags: {blue_print.tags}")

### Spawn a vehicle

`world.try_spawn_actor(...)` returns `None` if spawning fails (for example, if the spawn point is occupied).

Best practice: always check the result before calling methods such as `set_autopilot(True)` or `destroy()`.

Useful references:
- [`carla.World.try_spawn_actor`](https://carla.readthedocs.io/en/latest/python_api/#carla.World.try_spawn_actor)
- [`carla.Vehicle.set_autopilot`](https://carla.readthedocs.io/en/latest/python_api/#carla.Vehicle.set_autopilot)
- [`carla.Actor.destroy`](https://carla.readthedocs.io/en/latest/python_api/#carla.Actor.destroy)

### Actor exercises

1. Robust spawn: try 50 random spawn points and count successful/failed spawns.
2. Blueprint filtering: print only blueprints with `vehicle` in tags and choose one specific model.
3. Cleanup: create a script that safely destroys all vehicles spawned by your script.

In [35]:
vehicle = world.try_spawn_actor(blue_prints[0], spawn_points[0])
if vehicle is None:
    print("Spawn failed: spawn point occupied or invalid.")
else:
    print(f"Spawned vehicle: {vehicle.type_id}")

In [37]:
if vehicle is not None:
    vehicle.set_autopilot(True)
else:
    print("Autopilot not set: no vehicle available.")

In [ ]:
if vehicle is not None:
    vehicle.destroy()
    vehicle = None
    print("Vehicle destroyed.")
else:
    print("Nothing to destroy.")

## 5. Exercise Solutions

Run these cells in order. They are written to directly solve the exercises from sections 3 and 4.

### 5.1 Guided configuration exercise 

The code below implements the full guided scenario: connect, load map, set weather sequence, run 100 ticks in synchronous mode, then return to asynchronous mode.

In [ ]:
client = carla.Client("localhost", 2000)
client.set_timeout(15.0)

world = client.load_world("Town10HD_Opt")
print("Loaded Town10HD_Opt")

weather_sequence = [
    carla.WeatherParameters.CloudyNoon,
    carla.WeatherParameters.MidRainSunset,
    carla.WeatherParameters.HardRainNoon,
    carla.WeatherParameters.ClearSunset,
]

for weather in weather_sequence:
    world.set_weather(weather)
    print(f"Weather set to {weather}")
    time.sleep(4)

settings = world.get_settings()
try:
    settings.synchronous_mode = True
    world.apply_settings(settings)
    print("Switched to synchronous mode")

    for tick in range(100):
        world.tick()
        if (tick + 1) % 10 == 0:
            print(f"Tick {tick + 1}/100")
finally:
    settings = world.get_settings()
    settings.synchronous_mode = False
    world.apply_settings(settings)
    client.reload_world()
    print("Switched back to asynchronous mode and reloaded the world")

### 5.2 Additional exercise solution: timeout handling

In [ ]:
test_timeouts = [0.5, 5.0, 15.0]
test_maps = ["Town01", "Town02"]

for timeout in test_timeouts:
    client = carla.Client("localhost", 2000)
    client.set_timeout(timeout)

    print(f"\nTimeout = {timeout}s")
    for town in test_maps:
        try:
            client.load_world(town)
            print(f"  OK: load_world('{town}')")
        except RuntimeError as err:
            print(f"  FAIL: load_world('{town}') -> {err}")

### 5.3 Additional exercise solution: sync vs async comparison (10 seconds)

In [ ]:
client = carla.Client("localhost", 2000)
client.set_timeout(15.0)
world = client.get_world()

def measure_sync_ticks(world, seconds=10):
    settings = world.get_settings()
    original_sync = settings.synchronous_mode
    settings.synchronous_mode = True
    world.apply_settings(settings)

    ticks = 0
    start = time.time()
    try:
        while time.time() - start < seconds:
            world.tick()
            ticks += 1
    finally:
        settings = world.get_settings()
        settings.synchronous_mode = original_sync
        world.apply_settings(settings)

    return ticks

def measure_async_frames(world, seconds=10):
    settings = world.get_settings()
    original_sync = settings.synchronous_mode
    settings.synchronous_mode = False
    world.apply_settings(settings)

    start_frame = world.get_snapshot().frame
    time.sleep(seconds)
    end_frame = world.get_snapshot().frame

    settings = world.get_settings()
    settings.synchronous_mode = original_sync
    world.apply_settings(settings)

    return end_frame - start_frame

sync_ticks = measure_sync_ticks(world, seconds=10)
async_frames = measure_async_frames(world, seconds=10)

print(f"Sync mode:  {sync_ticks} ticks in 10 seconds")
print(f"Async mode: {async_frames} frames in 10 seconds")

### 5.4 Additional exercise solution: weather sequence function

In [ ]:
def apply_weather_sequence(world, presets, delay_seconds=3.0, cycles=2):
    """Apply weather presets in sequence. Set cycles=None for an endless loop."""
    cycle = 0
    while cycles is None or cycle < cycles:
        cycle += 1
        for weather in presets:
            world.set_weather(weather)
            print(f"[cycle {cycle}] Weather set to {weather}")
            time.sleep(delay_seconds)

client = carla.Client("localhost", 2000)
client.set_timeout(15.0)
world = client.get_world()

presets = [
    carla.WeatherParameters.ClearNoon,
    carla.WeatherParameters.CloudyNoon,
    carla.WeatherParameters.WetNoon,
    carla.WeatherParameters.SoftRainSunset,
]

apply_weather_sequence(world, presets, delay_seconds=2.0, cycles=1)

### 5.5 Actor exercise solutions

This section covers blueprint filtering, robust spawn attempts on 50 random points, and safe cleanup.

In [ ]:
import random

client = carla.Client("localhost", 2000)
client.set_timeout(15.0)
world = client.get_world()

vehicle_blueprints = world.get_blueprint_library().filter("vehicle.*")
print(f"Available vehicle blueprints: {len(vehicle_blueprints)}")

preferred_id = "vehicle.tesla.model3"
selected_blueprint = next(
    (bp for bp in vehicle_blueprints if bp.id == preferred_id),
    vehicle_blueprints[0]
)
print(f"Selected blueprint: {selected_blueprint.id}")

spawn_points = world.get_map().get_spawn_points()
num_trials = min(50, len(spawn_points))
selected_points = random.sample(spawn_points, num_trials)

spawned_vehicles = []
for point in selected_points:
    vehicle = world.try_spawn_actor(selected_blueprint, point)
    if vehicle is not None:
        spawned_vehicles.append(vehicle)

success_count = len(spawned_vehicles)
failure_count = num_trials - success_count
print(f"Spawn success: {success_count}")
print(f"Spawn failure: {failure_count}")

In [ ]:
if "spawned_vehicles" not in globals():
    spawned_vehicles = []

destroyed = 0
for vehicle in spawned_vehicles:
    if vehicle is not None and vehicle.is_alive:
        vehicle.destroy()
        destroyed += 1

print(f"Destroyed {destroyed} vehicles")
spawned_vehicles = []